## FIUBA CEIA Materia: NLP
# Desafio N.1
Alumno: Arias Suarez Federico
Este notebook es una copia de la notebook vista en clase, editada en las partes que se solicita al alumno realizar el desafio

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
import numpy as np
from sklearn.datasets import fetch_20newsgroups

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn -> este punto se hace mas abajo.
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto, veamos el primero
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

### Consigna del desafío 1

# **1**.
Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.


In [ ]:
# Obtener un índice aleatorio
indice_aleatorio = np.random.randint(0, len(newsgroups_train.data))
# Veamos similaridad de documentos. Tomemos algún documento

print(newsgroups_train.data[indice_aleatorio])

Hello world,

I want to write my Xt-application code like this:

{
    do_some_work();
    /* now I need some user input */
    XmCreateDialog();
    wait_for_user_input(input);
    if (input == "OK") {
       more_work();
    } else {
       other_work();
    }
}

So "more_work() and other_work()" are not in callback functions but the
application simply waits for the user to answer the question.

How can I code this in my Xt/Motif-application?

Thanks very much in advance.

Greetings, Huub.



In [ ]:
#El valor aleatorio nos entrega un documento que parece un correo acerca de un codigo de programacion.
#Veamos como esta clasificado:
print(y_train[indice_aleatorio])

5


In [ ]:
newsgroups_train.target_names[y_train[indice_aleatorio]]  #El tema de discucion es computacion windows x

'comp.windows.x'

Obtenemos los 5 documentos mas similares y analizamos:



In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cos_sim = cosine_similarity(X_train[indice_aleatorio], X_train)[0]

# los 5 documentos más similares:
most_sim = np.argsort(cos_sim)[::-1][1:6]

In [ ]:
#Veamos la clase de cada documento similar:
# y los 5 más similares son de las clases:
for i in most_sim:
  print(newsgroups_train.target_names[y_train[i]])

comp.windows.x
comp.windows.x
comp.windows.x
comp.windows.x
comp.os.ms-windows.misc


Vemos que 4 de 5 documentos son identicos en clase, el otro es similar(comp.os.ms-windows.misc) pero no identico
Veamos cada documento a ver de que se trata

In [ ]:
#Documento #1:
print(newsgroups_train.data[most_sim[0]])

Archive-name: x-faq/part5
Last-modified: 1993/04/04

----------------------------------------------------------------------
Subject: 119)  I'm writing a widget and can't use a float as a resource value.

Float resources are not portable; the size of the value may be larger than
the size of an XtPointer. Try using a pointer to a float instead; the Xaw
Scrollbar float resources are handled in this way. 

----------------------------------------------------------------------
Subject: 120)  Is this a memory leak in the X11R4 XtDestroyWidget()?!

Yes. This is the "unofficial" fix-19 for the X11R4 Destroy.c:

*** Destroy.c.1.37	Thu Jul 11 15:41:25 1991
--- lib/Xt/Destroy.c	Thu Jul 11 15:42:23 1991
***************
*** 1,4 ****
--- 1,5 ----
  /* $XConsortium: Destroy.c,v 1.37 90/09/28 10:21:32 swick Exp $ */
+ /* Plus unofficial patches in revisions 1.40 and 1.41 */
  
  /***********************************************************
  Copyright 1987, 1988 by Digital Equipment Corporation, Maynar

In [ ]:
#El documento tiene partes de codigo como bucles, lo cual demuestra similitud

In [ ]:
#Documento #2
print(newsgroups_train.data[most_sim[1]])

Archive-name: Xt-FAQ
Version: $Id: FAQ-Xt,v 1.28 93/04/02 12:41:12 ware Exp $

		    The X Toolkit Intrinsics F.A.Q
			  A monthly posting


This article contains the answers to some Frequently Asked Questions
(FAQ) from comp.windows.x about the X Toolkit Intrinsics.  To submit
questions (preferably with an answer) send email to: ware@cis.ohio-state.edu

Many FAQs, including this one, are available on the archive site
rtfm.mit.edu in the directory pub/usenet/news.answers.  The name
under which a FAQ is archived appears in the Archive-name 
line at the top of the article.  This FAQ is archived as Xt-FAQ.

All code fragments are public domain.  

			       Contents
0.  Xt Glossary
1.  Software Versions
2.  Related FAQ's
3.  Why does my application core dump when I use signals/alarms/cthreads?
4.  How do I use a different visual than the default?
5.  Which visual should an application use?
6.  Why do only Shell widgets have a Visual?
7.  Which visual, depth and colormap do Shells inherit?

In [ ]:
#El segundo documento tambien contiene partes de documentacion de codigo y explicaciones de aplicaciones, glosarios, etc.
#El documento contiene al inicio este texto:
'''This article contains the answers to some Frequently Asked Questions
(FAQ) from comp.windows.x about the X Toolkit Intrinsics.  To submit
questions (preferably with an answer) send email to'''

#Indicando que es para el sistema operativo comp.windows.x que es el mismo del documento base

In [ ]:
#Documento #3:
print(newsgroups_train.data[most_sim[2]])


I have the same problem.  I have looked at Motif++, WWL, InterViews, GINA++,
and a few variations on the above.  I've also done a cursory examination
of Rogue Wave's View.h++.  I like View.h++'s abstractions best of all of the
toolkits I mentioned, but the resulting code looks little like Motif, and I
have little confidence that this software will catch on or otherwise result
in significant longevity for my code.  GINA++ allows you to write code which
looks a great deal like Motif and also makes interesting use of inheritance,
but the resulting code is almost too Motif-like, and is certainly not
significantly less verbose than equivalent C code.  InterViews looks
promising, but I haven't found a free version with Motif support, and I'm
not confident how widely InterViews with Motif support will be adopted,
and what (if any) specific Motif support will be available over time.  The
other libraries produce code which is less Motif-like, but which does not make
sufficient use of the featu

Este documento a simple vista no es identico al documento base, tiene una similitud respecto a que es un correo, si se mira a detalle, el documento contiene palabras clave como C, C++ y lenguaje tecnico de programacion.
lo que brinda cierta similitud entre documentos, quizas porque la tematica esta mucho mas marcada que con otras clases.

In [ ]:
#Documento #4:
print(newsgroups_train.data[most_sim[3]])

I need to have PCs and SPARCstations run the same application ( namely
MicroSoft Project ). The original system ran on the PC. Now it needs to
be expanded to allow UNIX users to work with the application. The
current proposal is to use DESQview/X as a display server for the
application.

I would like to know your experiences with using DESQview/X to run an
application on a PC and displaying on a SPARCstation. I've heard that
the network traffic is slow.

Replies only by e-mail please.

Thanks, in advance.


Este documento tambien es muy parecido en el formato de correo electronico.
tambien contiene lenguaje relativo a programacion en UNIX, y sistemas de computacion

In [ ]:
#Documento #5 (ultimo):
print(newsgroups_train.data[most_sim[4]])

Hello.

      Is it possible to know minimize program manager when starting an
      application and to restore it when the application is ended ?
      If possible, please tell me how to do it !




Este ultimo documento tambien tiene similitudes en la forma de correo electronico, aunque la similitud cada vez es menor, ya que en este documento no se halla contenido tecnico de programacion

Como resumen de los 5 documentos, podemos ver que la similitud es menos marcada a medida que avanzamos en el numero de documento (del 1 al 5).

# **2**.
Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.


In [ ]:
#Primero pasamos el dataset a minusculas para que no haya problemas de case:
newsgroups_train.data = [doc.lower() for doc in newsgroups_train.data]

In [ ]:
###Ejemplo de la clase:
clf = MultinomialNB()
clf.fit(X_train, y_train)

# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

# el F1-score es una metrica adecuada para reportar desempeño de modelos de clasificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.6392404433573737

In [ ]:
#Vamos cambiar los parametros del vectorizador:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvect = TfidfVectorizer(max_features=16000, stop_words='english', min_df=3)
X_train = tfidfvect.fit_transform(newsgroups_train.data)


In [ ]:
#Ahora cambiamos los parametros del clasificador bayesiano:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=0.5, fit_prior=True)
clf.fit(X_train, y_train)

# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

f1_score(y_test, y_pred, average='macro')


0.6596264496782749

Vemos que ajustando:
- El vectorizador para tener max_features=17.000, sacando stopwords del ingles, y descartando terminos que aparecen menos de 15 veces.
- Agregando el alpha = 0.5 e instanciando el clasificador para que calcule la 'a priori'.

Logramos que el F1 score pase de 0.64 a 0.66. El f1 score sin hacer minusculas las palabras era de 0.58.

In [ ]:
#Vamos a probar ComplementNB y Gaussian NB:
from sklearn.naive_bayes import ComplementNB, GaussianNB

# Instanciar y entrenar Complement Naive Bayes
clf_complement = ComplementNB(alpha=0.5, fit_prior=True)
clf_complement.fit(X_train, y_train)

# Vectorizar el conjunto de prueba
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred_complement = clf_complement.predict(X_test)

# Calcular el F1-score para ComplementNB
f1_complement = f1_score(y_test, y_pred_complement, average='macro')
print("F1-score (ComplementNB):", f1_complement)

F1-score (ComplementNB): 0.6805816659818602


In [ ]:
#Gaussian NB:
# Para GaussianNB, debemos usar el conjunto de datos en forma densa
X_train_dense = X_train.toarray()

# Instanciar y entrenar Gaussian Naive Bayes
clf_gaussian = GaussianNB()
clf_gaussian.fit(X_train_dense, y_train)

# Vectorizar el conjunto de prueba
X_test_dense = X_test.toarray()
y_pred_gaussian = clf_gaussian.predict(X_test_dense)

# Calcular el F1-score para GaussianNB
f1_gaussian = f1_score(y_test, y_pred_gaussian, average='macro')
print("F1-score (GaussianNB):", f1_gaussian)


F1-score (GaussianNB): 0.49470509884948494


El mejor modelo es el **Complement NB (f1_score = 0.68)**, aunque el modelo basado en **Multinomial Naive Bayes (f1_score = 0.66)** presenta F1-score muy cercano al modelo multinomial.

# **3**.
Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


In [ ]:
# Transponer la matriz dispersa
X_train_transposed = X_train.transpose()

# Corroboramos
print("Tamaño de la matriz original:", X_train.shape)
print("Tamaño de la matriz transpuesta:", X_train_transposed.shape)

Tamaño de la matriz original: (11314, 7623)
Tamaño de la matriz transpuesta: (7623, 11314)


In [ ]:
#Ahora tenemos la matriz termino documento.
#definimos las palabras:

word_1 = 'phone'
word_2 = 'install'
word_3 = 'meal'
word_4 = 'wires'
word_5 = 'jesus'

In [ ]:
# Obtener el vocabulario
vocabulario = tfidfvect.get_feature_names_out()

#Generamos una funcion que devuelve las 5 palabras mas parececidas:
def get_similar_terms(word):
  # Obtener el índice de la palabra 'ball'
  indice_word = np.where(vocabulario == word)[0]

  if indice_word.size > 0:
      # Extraer el vector correspondiente a 'ball'
      vector_ball = X_train[:, indice_word].toarray().flatten()

      # Calcular la similitud del coseno con todas las demás palabras
      from sklearn.metrics.pairwise import cosine_similarity

      # Transponer la matriz para obtener un formato adecuado
      X_train_transposed = X_train.T

      # Calcular la similitud
      similitudes = cosine_similarity(vector_ball.reshape(1, -1), X_train_transposed).flatten()

      # Obtener las 5 palabras más similares
      indices_similares = np.argsort(similitudes)[-6:-1]  # Los últimos 6, excluyendo la última que es 'ball'

      # Mostrar las palabras más similares
      palabras_similares = vocabulario[indices_similares]
      similitudes_valores = similitudes[indices_similares]

      for palabra, similitud in zip(palabras_similares, similitudes_valores):
          print(f'Palabra: {palabra}, Similitud: {similitud:.4f}')
  else:
      print(f"La palabra {word} no se encuentra en el vocabulario.")

In [ ]:
get_similar_terms('president')  #word_1 = 'phone'

Palabra: clinton, Similitud: 0.2381
Palabra: oval, Similitud: 0.2637
Palabra: secretary, Similitud: 0.2829
Palabra: _________________________________________________________________, Similitud: 0.3362
Palabra: miyazawa, Similitud: 0.3757


Vemos que las palabras parecidas a phone son `presidente` son:
- `clinton`: referencia a un ex presidente de USA.
- `oval`: es la oficina donde trabaja el presidente de USA.
- `secretary`: puede referirse a un secretario presidencial.
- `____`: no es una palabra, es un error, que quizas aparece porque las cartas o mensajes del dataset contiene esa linea para denotar un estilo de mensaje.
- `miyazawa`: la relacion no es clara, pero parece referirse a una persona de origen japones.

In [ ]:
get_similar_terms('happy')  #word_2 = 'happy'

Palabra: om, Similitud: 0.1462
Palabra: interact, Similitud: 0.1551
Palabra: merchants, Similitud: 0.1736
Palabra: joy, Similitud: 0.1969
Palabra: birthday, Similitud: 0.3861


En este caso:
- `happy` tiene similitudes con las palabras propuestas por la matriz sparsa:

- `birthday`: relaciona la emocion del dia de cumpleaños. Alta similitud.
- `joy`: es sinonimo de happy.
- `merchants`: su relacion no es completamente clara.
- `interact`: su relacion no es completamente clara.
- `abroad`: Esta palabra tampoco tiene una relacion muy clara con `happy`

In [ ]:
get_similar_terms('meal')  #word_3 = 'meal'

Palabra: eat, Similitud: 0.1281
Palabra: inherent, Similitud: 0.1289
Palabra: recommendations, Similitud: 0.1755
Palabra: symptoms, Similitud: 0.1807
Palabra: bread, Similitud: 0.2931


- `bread`: es un tipo de meal. Relacion muy clara. es la que mas se 'parece'.
- `eat`: es el verbo que se usa con el sustantivo meal. La relacion es clara
- `inherit`: la relacion no es clara, no es claro heredar una comida.
- `recommendations`: si bien este sustantivo puede estar relacionado a cualquier otra cosa, se puede usar con el sustantivo meal, ya que las comidas se recomiendan.
- `symptoms`: no hay una relacion clara.


In [ ]:
get_similar_terms('wires')  #word_4 = 'wires'

Palabra: connectors, Similitud: 0.1340
Palabra: pushed, Similitud: 0.1357
Palabra: blank, Similitud: 0.1415
Palabra: wire, Similitud: 0.2171
Palabra: connecting, Similitud: 0.2224


- `connectors`: los cables (`wires`) se usan para conectar y pueden en ciertos casos requerir conectores.
- `pushed`: es un verbo que se puede combinar con `wires`, ya que se puede empujar un cable.
- `blank`: el Blank wire es un tipo de cable, generalmente de cobre.
- `wire`: es el singular
- `connecting`: accion que puede tener un cable, conectar.


In [ ]:
get_similar_terms('jesus')  #word_5 = 'jesus'

Palabra: bible, Similitud: 0.2004
Palabra: disciples, Similitud: 0.2070
Palabra: kingdom, Similitud: 0.2188
Palabra: god, Similitud: 0.2753
Palabra: christ, Similitud: 0.3095


Palabra `Jesus`:
- `Christ`: la relacion es clara, es un nombre propio Jesus Christ. Es la que mayor similitud tiene.
- `god`: Jesus es el hijo de Dios.
- `kingdom`: puede hacer referencia al reino de Dios/Jesus.
- `disciples`: Son los seguidores de Jesus.
- `biblia`: es el libro base de la religion de los seguidores de Jesucristo